In [1]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
# Metrics
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
# Text box
import ipywidgets as widgets
from IPython.display import display

/Users/aanderson/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Read-in movie lines and conversations datasets
movie_lines_path = '/Users/aanderson/Downloads/corpus/movie_lines.txt'
movie_conversations_path = '/Users/aanderson/Downloads/corpus/movie_conversations.txt'

In [3]:
lines = open(movie_lines_path, encoding='utf-8', errors='ignore').read().split('\n')
convers = open(movie_conversations_path, encoding='utf-8', errors='ignore').read().split('\n')

**Text Preprocessing**

In [4]:
class TextPreprocessor:
    def __init__(self, max_len=13):
        self.max_len = max_len
        self._compile_regex()

    def _compile_regex(self):
        self.regex_patterns = {
            r"i'm": "i am",
            r"he's": "he is",
            r"she's": "she is",
            r"that's": "that is",
            r"what's": "what is",
            r"where's": "where is",
            r"\'ll": " will",
            r"\'ve": " have",
            r"\'re": " are",
            r"\'d": " would",
            r"won't": "will not",
            r"can't": "cannot",
            r"[^\w\s]": "",
        }
        self.compiled_patterns = {re.compile(pattern): repl for pattern, repl in self.regex_patterns.items()}

    def clean_text(self, txt):
        txt = txt.lower()
        for pattern, repl in self.compiled_patterns.items():
            txt = pattern.sub(repl, txt)
        return txt

    def preprocess_data(self, convers, lines):
        exchange = [conver.split(' +++$+++ ')[-1][1:-1].replace("'", " ").replace(",", "").split() for conver in convers]
        diag = {line.split(' +++$+++ ')[0]: line.split(' +++$+++ ')[-1] for line in lines}
        questions, answers = self._extract_questions_answers(exchange, diag)
        return questions, answers

    def _extract_questions_answers(self, exchange, diag):
        questions, answers = [], []
        for conver in exchange:
            for i in range(len(conver) - 1):
                questions.append(diag.get(conver[i], ''))
                answers.append(diag.get(conver[i + 1], ''))
        sorted_ques = [q for q in questions if len(q.split()) < self.max_len]
        sorted_ans = [a for q, a in zip(questions, answers) if len(q.split()) < self.max_len]
        return sorted_ques, sorted_ans

In [5]:
# Apply preprocessing and cleaning
max_len = 13
preprocessor = TextPreprocessor(max_len=max_len)
sorted_ques, sorted_ans = preprocessor.preprocess_data(convers, lines)
clean_ques = [preprocessor.clean_text(q) for q in sorted_ques]
clean_ans = [preprocessor.clean_text(a) for a in sorted_ans]

# Trimming answers and lists
clean_ans = [' '.join(ans.split()[:11]) for ans in clean_ans]
clean_ans = clean_ans[:30000]
clean_ques = clean_ques[:30000]

In [6]:
# Before cleaning
print(sorted_ques[:5])
print(sorted_ans[:5])

["Well, I thought we'd start with pronunciation, if that's okay with you.", 'Not the hacking and gagging and spitting part.  Please.', "You're asking me out.  That's so cute. What's your name again?", 'Cameron.', 'Why?']
['Not the hacking and gagging and spitting part.  Please.', "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?", 'Forget it.', "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.", 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.']


In [7]:
# After cleaning
print(clean_ques[:5])
print(clean_ans[:5])

['well i thought we would start with pronunciation if that is okay with you', 'not the hacking and gagging and spitting part  please', 'you are asking me out  that is so cute what is your name again', 'cameron', 'why']
['not the hacking and gagging and spitting part please', 'okay then how bout we try out some french cuisine saturday', 'forget it', 'the thing is cameron i am at the mercy of a', 'unsolved mystery she used to be really popular when she started']


In [8]:
# Counting word occurrences and creating vocab based on threshold
word2count = {}
for line in clean_ques + clean_ans:
    for word in line.split():
        word2count[word] = word2count.get(word, 0) + 1

# Creating the vocabulary based on threshold
thresh = 5
vocab = {word: num for num, (word, count) in enumerate(word2count.items()) if count >= thresh}

In [9]:
# Adding special tokens to the vocabulary
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
for token in tokens:
    vocab[token] = len(vocab)

# Creating the inverse vocabulary
inv_vocab = {num: word for word, num in vocab.items()}

# Encoding the questions and answers
encoder_inp = [[vocab.get(word, vocab['<OUT>']) for word in line.split()] for line in clean_ques]
decoder_inp = [[vocab.get(word, vocab['<OUT>']) for word in ('<SOS> ' + line + ' <EOS>').split()] for line in clean_ans]

# Sample output for verification
encoder_inp[:5], decoder_inp[:5]


([[0, 1, 2, 3, 4, 5, 6, 4088, 8, 9, 10, 11, 6, 12],
  [13, 14, 4088, 16, 4088, 16, 4088, 19, 20],
  [12, 21, 22, 23, 24, 9, 10, 25, 26, 27, 10, 28, 29, 30],
  [31],
  [32]],
 [[4089, 13, 14, 4088, 16, 4088, 16, 4088, 19, 20, 4087],
  [4089, 11, 93, 56, 1129, 3, 738, 24, 361, 283, 4088, 141, 4087],
  [4089, 166, 159, 4087],
  [4089, 14, 140, 10, 31, 1, 103, 133, 14, 3364, 81, 38, 4087],
  [4089, 4088, 8415, 79, 271, 75, 125, 430, 945, 365, 79, 1107, 4087]])

In [10]:
# Convert lists to PyTorch tensors
encoder_inp_tensors = [torch.LongTensor(seq) for seq in encoder_inp]
decoder_inp_tensors = [torch.LongTensor(seq) for seq in decoder_inp]

# Padding
max_seq_len = max(max(len(seq) for seq in encoder_inp_tensors), max(len(seq) for seq in decoder_inp_tensors))
encoder_inp_tensors_padded = [torch.cat([seq, torch.LongTensor([vocab['<PAD>']] * (max_seq_len - len(seq)))]) for seq in encoder_inp_tensors]
decoder_inp_tensors_padded = [torch.cat([seq, torch.LongTensor([vocab['<PAD>']] * (max_seq_len - len(seq)))]) for seq in decoder_inp_tensors]

# Convert lists of tensors to 2D tensors
encoder_inp_padded = torch.stack(encoder_inp_tensors_padded, dim=0)
decoder_inp_padded = torch.stack(decoder_inp_tensors_padded, dim=0)
decoder_final_output = decoder_inp_padded[:, 1:]

In [11]:
print(encoder_inp_padded.shape)
print(decoder_inp_padded.shape)
print(decoder_final_output.shape)


torch.Size([30000, 15])
torch.Size([30000, 15])
torch.Size([30000, 14])


In [12]:
decoder_final_output.shape

torch.Size([30000, 14])

In [13]:
# Verify
encoder_inp_padded.shape, decoder_inp_padded.shape, decoder_final_output.shape

(torch.Size([30000, 15]), torch.Size([30000, 15]), torch.Size([30000, 14]))

In [14]:
encoder_inp_padded[22]

tensor([  93,    9,   10,   94,   12,   95,   75,   96, 4086, 4086, 4086, 4086,
        4086, 4086, 4086])

In [15]:
test_batch = torch.tensor(encoder_inp_padded[2].to(dtype=torch.long))
test_batch2 = torch.tensor(encoder_inp_padded[1].to(dtype=torch.long))
batch_tester = torch.stack([test_batch.T,test_batch2])

/var/folders/zr/tz5czcmd56nghjwmrkfjvrr80000gn/T/ipykernel_9007/1393693452.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_batch = torch.tensor(encoder_inp_padded[2].to(dtype=torch.long))
/var/folders/zr/tz5czcmd56nghjwmrkfjvrr80000gn/T/ipykernel_9007/1393693452.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_batch2 = torch.tensor(encoder_inp_padded[1].to(dtype=torch.long))
/var/folders/zr/tz5czcmd56nghjwmrkfjvrr80000gn/T/ipykernel_9007/1393693452.py:3: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torc

**Encoder, Decoder and Seq2Seq**

In [16]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        embedded = self.embedding(input_seq)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths.cpu(), enforce_sorted=False)
        outputs, hidden = self.gru(packed, hidden)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:]
        return outputs, hidden

In [17]:
class Decoder(nn.Module):
    def __init__(self, method, hidden_size):
        super(Decoder, self).__init__()
        self.method = method
        self.hidden_size = hidden_size
        
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
            
    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        
        attn_energies = attn_energies.t()
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [18]:
input_size = len(vocab)
src_vocab_size = len(vocab)
trg_vocab_size = len(vocab)
hidden_size = 200 

encoder = Encoder(input_size, hidden_size)

test_input = encoder_inp_padded[1].unsqueeze(1)
test_input_lengths = torch.tensor([len(test_input)]) 

# Double checking sequence length:
assert test_input.shape[0] >= torch.max(test_input_lengths), "Declared sequence length exceeds actual length"

# Encoder
output, hidden = encoder(test_input, test_input_lengths)

In [19]:
class TransformerSeq2Seq(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers):
        super(TransformerSeq2Seq, self).__init__()

        # Encoder
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead),
            num_layers=num_encoder_layers
        )

        # Decoder
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model, nhead),
            num_layers=num_decoder_layers
        )

        # Embeddings
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.trg_embedding = nn.Embedding(trg_vocab_size, d_model)

        # Output layer
        self.output_layer = nn.Linear(d_model, trg_vocab_size)

    def forward(self, src, trg):
        # Embed source and target sequences
        src_embedded = self.src_embedding(src)
        trg_embedded = self.trg_embedding(trg)

        # Encode source sequence
        encoder_output = self.encoder(src_embedded)

        # Decode target sequence with encoder output
        decoder_output = self.decoder(trg_embedded, encoder_output)

        # Final prediction
        output = self.output_layer(decoder_output)

        return output

In [20]:
EPOCHS = 2
LR = 0.001

# Defining parameters
d_model = 512
nhead = 8
num_encoder_layers = 6
num_decoder_layers = 6

model = TransformerSeq2Seq(src_vocab_size, trg_vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers)

criterion = nn.CrossEntropyLoss(ignore_index=vocab['<PAD>'])
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for epoch in range(EPOCHS):
    data_loader = DataLoader(
        list(zip(encoder_inp_padded, decoder_inp_padded, decoder_final_output)),
        batch_size=3,
        shuffle=True
    )
    for i, (encoder_input, decoder_input, decoder_output) in enumerate(data_loader):
        optimizer.zero_grad()
        
        if (encoder_input >= src_vocab_size).any() or (decoder_input >= trg_vocab_size).any():
            continue

        outputs = model(encoder_input, decoder_input)
        
        loss = criterion(outputs[:, 1:].contiguous().view(-1, outputs.shape[-1]), decoder_output.view(-1))
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f"Epoch [{epoch + 1}/{EPOCHS}], Loss: {loss.item():.4f}")

Epoch [1/2], Loss: 6.1925
Epoch [1/2], Loss: 5.1615
Epoch [1/2], Loss: 5.8120
Epoch [1/2], Loss: 4.7357
Epoch [1/2], Loss: 5.3043
Epoch [1/2], Loss: 4.6702
Epoch [1/2], Loss: 5.9605
Epoch [1/2], Loss: 5.8685
Epoch [1/2], Loss: 5.3909
Epoch [1/2], Loss: 5.7470
Epoch [1/2], Loss: 5.3402
Epoch [1/2], Loss: 5.0852
Epoch [1/2], Loss: 5.5482
Epoch [1/2], Loss: 5.4563
Epoch [1/2], Loss: 5.8816
Epoch [1/2], Loss: 5.0328
Epoch [1/2], Loss: 4.5945
Epoch [1/2], Loss: 6.1544
Epoch [1/2], Loss: 5.7120
Epoch [1/2], Loss: 4.5364
Epoch [1/2], Loss: 5.5490
Epoch [1/2], Loss: 5.0505
Epoch [1/2], Loss: 5.0775
Epoch [1/2], Loss: 5.9997
Epoch [1/2], Loss: 5.6600
Epoch [1/2], Loss: 4.6193
Epoch [2/2], Loss: 5.2666
Epoch [2/2], Loss: 4.7714
Epoch [2/2], Loss: 5.0579
Epoch [2/2], Loss: 4.8656
Epoch [2/2], Loss: 4.3671
Epoch [2/2], Loss: 5.0789
Epoch [2/2], Loss: 5.1490
Epoch [2/2], Loss: 4.7253
Epoch [2/2], Loss: 5.2326
Epoch [2/2], Loss: 5.7042
Epoch [2/2], Loss: 4.2576
Epoch [2/2], Loss: 6.0411
Epoch [2/2],

In [24]:
# Save the model
torch.save(model.state_dict(), 'transformer_model.pth')

In [ ]:
from flask import Flask, request, jsonify
import torch

app = Flask(__name__)

# Load the saved model
model = TransformerSeq2Seq(src_vocab_size, trg_vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers)
model.load_state_dict(torch.load('transformer_model.pth'))
model.eval()

@app.route('/predict', methods=['POST'])
def predict():
    input_data = request.json  # Assuming you send data in JSON format
    # Preprocess input_data if needed
    # Perform inference using the model
    # Return the model's predictions

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.0.205:8080
Press CTRL+C to quit
